# Deep fake

Infelizmente deep fake tem um potencial destrutivo. meu warning é importante. Você pode usar este notebook apenas com imagens de quem te deu autorização explícita para fazer os testes de deep fake, sem exceção. Grandes poderes, grandes responsabilidades. Queremos informar, ensinar, aprender.

Caso forke esse notebook, siga as regras de uso.



Automatizado por Guilherme Silveira, https://twitter.com/guilhermecaelum

Original de https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb "First Order Motion Model for Image Animation"


### PASSO 1

Se cadastre e baixe o arquivo vox-cpk.pth.tar se cadastrando aqui e baixando https://yadi.sk/d/lEw8uRm140L_eQ
Ou então em https://mega.nz/file/59JknSyY#R1jZr0MIb8rfpoQCX787AVcUPT96uTpfA5GdlrpL7vM

### PASSO 2

Crie um diretório chamado `deepfake` no seu google drive.

Coloque lá:
- uma imagem com `nome.png`
- o vídeo com `nome.mp4`
- o arquivo `vox-cpk.pth.tar`

In [ ]:
# Definir Diretório Raiz

import os
local = os.getcwd()
print('Seu diretório raiz esta localizado em:', local)

### PASSO 3

In [ ]:
# coloque aqui o nome do arquivo da sua imagem e a extensão

base_name = "image"
image_extension = "png"

### PASSO 4

In [ ]:
# PASSO 4

# corrija o corte do vídeo de origem. rode uma vez assim, depois vá atualizando

# assumindo que o vídeo é 1920 x 1080
# o rosto da pessoa está na parte "de baixo" do vídeo

# Se seu video estiver nas dimenções 256 x 256 não será necessario um corte

DIMENSION = 256
MIN_X = 0
MIN_Y = 0

should_crop = True

## Setup de diretórios

**Git clone**

In [ ]:
# O processo abaixo pode levar alguns segundos, pois os mesmos estão sendo clonados de um repositório
!git clone https://github.com/AliaksandrSiarohin/first-order-model

In [ ]:
cd first-order-model

# Pequenas alterações no vídeo original

In [ ]:
movie_input = base_name
movie_original = f"{local}\{movie_input}.mp4"
print(movie_original)

In [ ]:
input = base_name
image_output = f"{local}\{input}-256.png"
print(image_output)

## Conferindo dimensões

In [ ]:
import warnings


warnings.filterwarnings("ignore")

In [ ]:
# Install Pillow
!pip install pip
!pip install --upgrade pip
!pip install --upgrade Pillow

In [ ]:
print('Diretório atual: '+os.getcwd() )

In [ ]:
from PIL import Image


im = Image.open(f"../{input}.{image_extension}")
width, height = im.size

if width != height:
  new_size = min(width, height)
  left = (width - new_size)/2
  top = (height - new_size)/2
  right = (width + new_size)/2
  bottom = (height + new_size)/2

  cut = (left, top, right, bottom)
  print(cut)
  im = im.crop(cut)
  print(f"Oh, a imagem que você pegou não é quadrada, cortando o meio... {width} x {height}")


if width < 256:
  raise Exception(f"Oh, a imagem que você pegou é pequena, vai zoar! {width}")

print(im.size)
im.thumbnail((256, 256))
print(im.size)
im.save(image_output)
display(im)

In [ ]:
# Install moviepy
!pip install moviepy

In [ ]:
# FFMPEG
# Encontre uma ferramenta de FFMPEG e clone para dentro do diretorio raiz

In [ ]:
import os
import shutil
from moviepy.editor import VideoFileClip

movie_source = "../movie_quadrado.mp4"

if should_crop:
  !ffmpeg -y -hide_banner -loglevel warning -i "{movie_original}" -filter:v "crop={DIMENSION}:{DIMENSION}:{MIN_X}:{MIN_Y}" "$movie_source"
else:
  shutil.copyfile(movie_original, movie_source)

In [ ]:
from moviepy.editor import ipython_display


ipython_display(movie_source, width=256)

## Gerando mp3 e alterando tamanhos

In [ ]:
generated_audio = f"../generated_audio_{base_name}.mp3"
!ffmpeg -y -hide_banner -loglevel warning -i "$movie_original" -vn -ar 44100 -ac 2 -ab 192k -f mp3 "$generated_audio"

In [ ]:
# Install scikit-image
!pip install scikit-image

# Widgets
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
# TQDM - PROGRESS BAR

!pip install tqdm
os.system('pip install -e git+https://github.com/tqdm/tqdm.git@master#egg=tqdm')

In [ ]:
import imageio
from skimage.transform import resize
from tqdm.notebook import trange, tqdm

source_image = imageio.imread(image_output)
driving_video = imageio.mimread(movie_source, memtest=False)

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in tqdm(driving_video, desc="video resize")]

In [ ]:
!pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


def display_side_by_side(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in tqdm(range(len(driving)), "generating slides"):
        cols = [source]
        cols.append(driving[i])
        if generated:
        # if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat=False)
    plt.close()
    return HTML(ani.to_html5_video())


display_side_by_side(source_image, driving_video)

# O deepfake

In [ ]:
# se a célula a seguir não funcionar, descomente a linha a seguir
# por exemplo, se você usa o google colab pro, você talvez
# tenha que definir qual device de GPU será utilizado, por isso ;)

CUDA_VISIBLE_DEVICES=0

In [ ]:
# Outro possível problema
# Caso você não tenha o GPU ativado por padrão, você pode ir no menu Runtime > Change runtime type > GPU

# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
#   print('and then re-execute this cell.')
# else:
#   print(gpu_info)

In [ ]:
!pip install pyyaml

In [ ]:
# Acesso o seguinte diretorio

import os
os.chdir(local+'/first-order-model')
print('Diretório atual: '+os.getcwd() )

In [ ]:
# A Instalação do pacote abaixo pode demorar um pouco, na verdade muito!
# Você pode realizar a instalação utulizando o terminal do miniconda ou conda, para acompanhar o progresso

!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch -c=conda-forge
# !conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
# !pip install torch==1.8.0+cu102 torchvision==0.9.0+cu102 torchaudio===0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# Pacotes necessarios para rodar o proximo algoritimo

!pip install -U scikit-learn
!pip install pandas

In [ ]:
import yaml
import torch

from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
#generator, kp_detector = load_checkpoints(config_path='C:/Users\elias\Desktop\DeepFaker\first-order-model\config\vox-256.yaml',
                            checkpoint_path='C:/Users\elias\Desktop\DeepFake/vox-cpk.pth.tar')

## Animando

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte
from skimage.io import concatenate_images

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

display_side_by_side(source_image, driving_video, predictions)

In [ ]:
def frames_to_frame(f1, f2,f3):
  return np.concatenate((f1, np.array(f2), np.array(f3)),axis=1)

In [ ]:
imageio.mimsave('../generated.mp4', [img_as_ubyte(frames_to_frame(source_image,f2,f3)) for f2,f3 in tqdm(zip(driving_video, predictions), desc="concatenating frames")])

display_side_by_side(source_image, driving_video, predictions)

In [ ]:
!pip install files

In [ ]:
import files

# files.download('../generated.mp4')
ipython_display("../generated.mp4")

## Definindo a velocidade

In [ ]:
from moviepy.editor import VideoFileClip
generated_duration = VideoFileClip("../generated.mp4").duration
original_duration = VideoFileClip(movie_source).duration
multiple = generated_duration / original_duration
print( generated_duration, original_duration, multiple )

## O vídeo final

In [ ]:
#CASO O VIDEO ORIGINAL NÃO POSSUA AUDIO, PODE OCORRER UM ERRO, NÃO SE PREOCUPE!

setpts = f'setpts=PTS/{multiple}'
!ffmpeg -y -hide_banner -loglevel warning -i "../generated.mp4" -filter:v $setpts "../generated_speed_control.mp4"
print(f"Colocando o áudio")

final_name = f"../generated_final_{base_name}.mp4"
!ffmpeg -y -hide_banner -loglevel warning -i "../generated_speed_control.mp4" -i $generated_audio -c:v copy -c:a aac $final_name

ipython_display(final_name)

## Demonstração de keypoint relativos

Estilo de cabelo etc serão copiados

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
display_side_by_side(source_image, driving_video, predictions)

## Download

In [ ]:
import files
files.download(final_name)